In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
mpl.rcParams["font.family"] = 'NanumBarunGothic'
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib inline

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
#삼성전자 파일 업로드
from google.colab import files
uploaded=files.upload()

In [ ]:
import io
data=pd.read_csv(io.BytesIO(uploaded['SSE2015.csv']))
df=data
df.head()

In [ ]:
idx = df.index[df['Date'] <= '2018-04-30'][-1]
df['Volume'].iloc[:idx] = df['Volume'].iloc[:idx]*50

In [ ]:
index = df[df['Open'] <= 0].index
df.drop(index, inplace=True)

In [ ]:
df['5days']=df['Close'].rolling(window=5).mean()
df['10days']=df['Close'].rolling(window=10).mean()
df['20days']=df['Close'].rolling(window=20).mean()
df['60days']=df['Close'].rolling(window=60).mean()
df['120days']=df['Close'].rolling(window=120).mean()
df.head()

In [ ]:
#df3:NaN을 0으로 바꾼 데이터
df3=df
df3.fillna(0, inplace=True)
df3.head()

In [ ]:
df3.set_index('Date',inplace=True)

In [ ]:
scaler=MinMaxScaler()
scale_cols=['Open','High','Low','Close','Volume','Change','5days','10days','20days','60days','120days']
df_scaled=scaler.fit_transform(df[scale_cols])

In [ ]:
df_scaled=pd.DataFrame(df_scaled)
df_scaled.columns=scale_cols
print(df_scaled)

In [ ]:
TEST_SIZE=200
train = df_scaled[:-TEST_SIZE]
test = df_scaled[-TEST_SIZE:]

In [ ]:
def make_dataset(data, label, window_size=20):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [ ]:
feature_cols = ['Open', 'High', 'Low', 'Volume','5days','10days','20days','60days','120days']
label_cols = ['Close']

train_feature = train[feature_cols]
train_label = train[label_cols]

train_feature, train_label = make_dataset(train_feature, train_label, 20)

from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

x_train.shape, x_valid.shape

In [ ]:
test_feature=test[feature_cols]
test_label=test[label_cols]

test_feature, test_label = make_dataset(test_feature, test_label, 20)
test_feature.shape, test_label.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(16, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               return_sequences=False)
          )
model.add(Dense(1))

In [ ]:
model.compile(loss='mse',optimizer='adam',metrics=['mse'])

from keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(patience=20)
model.fit(x_train,y_train,epochs=100,batch_size=1,validation_split=0,callbacks=[early_stopping])

In [ ]:
pred=model.predict(test_feature)

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(test_label, label='actual')
plt.plot(pred, label='prediction')
plt.legend()
plt.show()

In [ ]:
trainScore = model.evaluate(x_train, y_train, batch_size=1, verbose=0)
model.reset_states()
print('Train Score: ', trainScore)
valScore = model.evaluate(x_valid, y_valid, batch_size=1, verbose=0)
model.reset_states()
print('Validataion Score: ', valScore)

In [ ]:
mse=mean_squared_error(test_label, pred)
rmse=np.sqrt(mse)
rmse